In [66]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime
import pandas.tseries.offsets as of  # for timeoffset

In [ ]:
'''
# 1주전 lastday
target_day_list + of.Week(n= -1, weekday = 4)
# 1달전 lastday
target_day_list + of.MonthEnd(n= -1)
# 1분기전 lastday
target_day_list + of.QuarterEnd(n= -1)
# 1년전 lastday
target_day_list + of.YearEnd(n= -1)
'''

In [67]:
def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [68]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v3.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
    
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

#5. target day 확인
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")

#(1) 월
#interval calculate before 12 month , before 1 month
init_month_timestamp = target_day_list + of.MonthEnd(n= -12)
end_month_timestamp = target_day_list + of.MonthEnd(n= -1)
fin_result['init_month'] = init_month_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_month'] = end_month_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

#(2) 주
#interval calculate before 30 weeks , before 1 weeks
init_week_timestamp = target_day_list + of.Week(n= -30, weekday = 4)
end_week_timestamp = target_day_list + of.Week(n= -1, weekday = 4)
fin_result['init_week'] = init_week_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_week'] = end_week_timestamp.apply(lambda x: x.strftime("%Y%m%d"))


#(3) 일
check_init_timestamp = target_day_list - datetime.timedelta(days=60)
fin_result['check_init_day'] = check_init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

#for 크로스전 for 3번째전략
check_init_timestamp2 = target_day_list - datetime.timedelta(days=30)
fin_result['check_init_day2'] = check_init_timestamp2.apply(lambda x: x.strftime("%Y%m%d"))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [69]:
fin_result = fin_result[fin_result['target_m']>='201909'].reset_index()
del fin_result['index']
fin_result['index'] = fin_result.index

In [70]:
fin_result.columns

Index(['interval_end', 'shcode', 'target_day', 'target_m', 'shname',
       'init_month', 'end_month', 'init_week', 'end_week', 'check_init_day',
       'check_init_day2', 'index'],
      dtype='object')

In [92]:
i = 560
v = fin_result['shcode'].iloc[i]
#(1) 월봉대세판단
df_monthly = query.query_t1305_month(shcode = fin_result['shcode'].iloc[i],
                                     fr = fin_result['init_month'].iloc[i],
                                     to = fin_result['end_month'].iloc[i])
'''
#except1: data없는경우
if df_monthly.shape[0] == 0:
    ma5 = None
    month_ma5_score = None
    high_month = None
    for_net_buy_sum = None
    com_net_buy_sum = None
    per_net_buy_sum = None   
    pass
else:
    # ma preprocesser
    df_monthly = preprocesser.sma_monthly(df_monthly)
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]

    # 대세 판단1 : 월봉 5이평이 감소하지않음ㄴ
    ma5 = non_decreasing(df_monthly['close_ma5'])[-5:]
    month_ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
    high_month = df_monthly['high'].max()
    for_net_buy_sum = df_monthly['for_net_buy'].sum()
    com_net_buy_sum = df_monthly['com_net_buy'].sum()
    per_net_buy_sum = df_monthly['per_net_buy'].sum()
'''
#(2) 주봉대세판단
df_weekly = query.query_t1305_week(shcode = fin_result['shcode'].iloc[i],
                                     fr = fin_result['init_week'].iloc[i],
                                     to = fin_result['end_week'].iloc[i])
#except1: data없는경우
#if df_weekly.shape[0] ==0:
#    continue
# ma preprocesser
df_weekly = preprocesser.sma_weekly(df_weekly)
amount = [0.15, 0.15, 0.2, 0.2, 0.3]

# 대세 판단1 : 주봉 5이평이 감소하지않음
ma5 = non_decreasing(df_weekly['close_ma5'])[-5:]
week_ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)

# 대세 판단2 : 주봉 20이평이 감소하지않음
ma20 = non_decreasing(df_weekly['close_ma20'])[-5:]
week_ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)

# 대세 판단3 : 주별확인
# 1. 위에 1-1 위에stay, 1-2 데크, 2,아래  2-1 아래스테이 2-2. 골크
close_ma5 = df_weekly['close_ma5'].iloc[-5:]
close_ma20 = df_weekly['close_ma20'].iloc[-5:]

prev_close_ma5 = df_weekly['close_ma5'].shift(1).iloc[-5:]
prev_close_ma20 = df_weekly['close_ma20'].shift(1).iloc[-5:]

week_golden_cross = np.where(((close_ma5 >= close_ma20) & (prev_close_ma5 <= prev_close_ma20)),1,0).sum() # gc
week_dead_cross = np.where(((close_ma20 <= close_ma20) & (prev_close_ma5 >= prev_close_ma20)),1,0).sum() # dc
week_up = np.where((close_ma5 >= close_ma20), 1, 0).sum() # 정배열
week_down = np.where((close_ma5 < close_ma20), 1, 0).sum() # 역배열



In [93]:
df_weekly.tail()

,date,open,high,low,close,volume,deal_f,burn_ratio,rotate_ratio,for_net_buy,com_net_buy,per_net_buy,close_ma5,close_ma20
25,20190802,20250,20800,18350,19650,602,80.19,2.69,10.65,-17796,-5127,22989,20760.0,17967.5
26,20190809,19400,19600,16050,18400,599,77.49,3.28,10.60,33473,-1330,-41300,20120.0,18080.0
27,20190816,18700,19800,16550,18500,727,96.25,2.41,12.86,-47706,64771,-19564,19560.0,18285.0
28,20190823,18900,20650,18450,19850,658,101.99,1.41,11.63,-57586,52846,-4872,19340.0,18532.5
29,20190830,19050,19850,18200,19750,774,71.38,1.36,13.68,-7835,-37697,39707,19230.0,18735.0


In [94]:
np.where(((close_ma5 >= close_ma20) & (prev_close_ma5 <= prev_close_ma20)),1,0) # gc


array([0, 0, 0, 0, 0])

In [98]:
(close_ma20 <= close_ma20)

25    True
26    True
27    True
28    True
29    True
Name: close_ma20, dtype: bool

In [95]:
np.where(((close_ma20 <= close_ma20) & (prev_close_ma5 >= prev_close_ma20)),1,0) # dc

array([1, 1, 1, 1, 1])

In [96]:
np.where((close_ma5 >= close_ma20), 1, 0) # 정배열

array([1, 1, 1, 1, 1])

In [97]:
np.where((close_ma5 < close_ma20), 1, 0) # 역배열

array([0, 0, 0, 0, 0])

In [99]:
per_df = pd.DataFrame([],columns = ['date','index','shcode'])

#for i in range(len(result_for_draw)):
i = 382
df = query.query_ohlcv(shcode = result_for_draw['shcode'].iloc[i],
                     fr=result_for_draw['interval_init'].iloc[i],
                     to=result_for_draw['interval_end'].iloc[i])

#데이터전처리
df = preprocesser.sma_long(df)
df = preprocesser.obv(df)
df = preprocesser.rsi(df)
df = preprocesser.stochastic(df)
df = preprocesser.marking_cross_sma(df)
df = preprocesser.marking_cross_rsi(df)
df = preprocesser.marking_cross_stochastic(df)

#df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
df['index'] = result_for_draw['index'].iloc[i]
df['shcode'] = result_for_draw['shcode'].iloc[i]
df['target_day'] = result_for_draw['target_day'].iloc[i]

per_df = per_df.append(df[['date','index','shcode',
                           'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
                           'obv',
                           'rsi14', 'rsi14_signal', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
                           'stoc_g_c', 'stoc_d_c']])

per_df[['date','index','shcode',
                           'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
                           'obv',]].iloc[10:]

In [71]:
rrr = pd.DataFrame([],columns = ['date',  'close_ratio','open_', 'low', 'high','close','volume' ,
                                 'range_position', 'squeezed',  'shcode','shname', 'target_m', 'index',
                                 'month_ma5_score', 'high_month', 'for_net_buy_sum','com_net_buy_sum', 'per_net_buy_sum',
                                 'week_ma5_score','week_ma20_score',
                                 'week_golden_cross', 'week_dead_cross', 'week_up', 'week_down'
                                ])

In [72]:
for i,v in fin_result['shcode'].items():
#i = 0
#v = fin_result['shcode'].iloc[i]
    #(1) 월봉대세판단
    df_monthly = query.query_t1305_month(shcode = fin_result['shcode'].iloc[i],
                                         fr = fin_result['init_month'].iloc[i],
                                         to = fin_result['end_month'].iloc[i])

    #except1: data없는경우
    if df_monthly.shape[0] == 0:
        ma5 = None
        month_ma5_score = None
        high_month = None
        for_net_buy_sum = None
        com_net_buy_sum = None
        per_net_buy_sum = None   
        pass
    else:
        # ma preprocesser
        df_monthly = preprocesser.sma_monthly(df_monthly)
        amount = [0.15, 0.15, 0.2, 0.2, 0.3]

        # 대세 판단1 : 월봉 5이평이 감소하지않음ㄴ
        ma5 = non_decreasing(df_monthly['close_ma5'])[-5:]
        month_ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
        high_month = df_monthly['high'].max()
        for_net_buy_sum = df_monthly['for_net_buy'].sum()
        com_net_buy_sum = df_monthly['com_net_buy'].sum()
        per_net_buy_sum = df_monthly['per_net_buy'].sum()

    #(2) 주봉대세판단
    df_weekly = query.query_t1305_week(shcode = fin_result['shcode'].iloc[i],
                                         fr = fin_result['init_week'].iloc[i],
                                         to = fin_result['end_week'].iloc[i])
    #except1: data없는경우
    if df_weekly.shape[0] ==0:
        continue
    # ma preprocesser
    df_weekly = preprocesser.sma_weekly(df_weekly)
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]

    # 대세 판단1 : 주봉 5이평이 감소하지않음
    ma5 = non_decreasing(df_weekly['close_ma5'])[-5:]
    week_ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)

    # 대세 판단2 : 주봉 20이평이 감소하지않음
    ma20 = non_decreasing(df_weekly['close_ma20'])[-5:]
    week_ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)

    # 대세 판단3 : 주별확인
    # 1. 위에 1-1 위에stay, 1-2 데크, 2,아래  2-1 아래스테이 2-2. 골크
    close_ma5 = df_weekly['close_ma5'].iloc[-5:]
    close_ma20 = df_weekly['close_ma20'].iloc[-5:]

    prev_close_ma5 = df_weekly['close_ma5'].shift(1).iloc[-5:]
    prev_close_ma20 = df_weekly['close_ma20'].shift(1).iloc[-5:]

    week_golden_cross = np.where(((close_ma5 >= close_ma20) & (prev_close_ma5 <= prev_close_ma20)),1,0).sum() # gc
    week_dead_cross = np.where(((close_ma5 <= close_ma20) & (prev_close_ma5 >= prev_close_ma20)),1,0).sum() # dc
    week_up = np.where((close_ma5 >= close_ma20), 1, 0).sum() # 정배열
    week_down = np.where((close_ma5 < close_ma20), 1, 0).sum() # 역배열

    #(3) 일별확인
    df = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                            fr = fin_result['check_init_day2'].iloc[i], 
                            to = fin_result['target_day'].iloc[i])

    #종가상승률
    close_ratio_list = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])]
    close_ratio_list.insert(0,0)
    test_list = pd.DataFrame(dict(date = np.array(df)[:,0],
                                  close_ratio = close_ratio_list,
                                  open_ = df['open'],
                                  low = df['low'],
                                  high = df['high'],
                                  close = df['close'],
                                  volume = df['volume'],
                                  range_position = (df['close']-df['low'])/(df['high']-df['low']),
                                  squeezed = abs(df['close'] /df['open'] - 1) <= 0.05
                                 ))

    test_list['shcode'] = fin_result['shcode'].iloc[i]
    test_list['shname'] = fin_result['shname'].iloc[i]
    test_list['target_m'] = fin_result['target_m'].iloc[i]
    test_list['index'] = i

    test_list['month_ma5_score'] = month_ma5_score

    test_list['high_month'] = high_month
    test_list['for_net_buy_sum'] = for_net_buy_sum
    test_list['com_net_buy_sum'] = com_net_buy_sum
    test_list['per_net_buy_sum'] = per_net_buy_sum

    test_list['week_ma5_score'] = week_ma5_score
    test_list['week_ma20_score'] = week_ma20_score
    
    test_list['week_golden_cross'] = week_golden_cross
    test_list['week_dead_cross'] =  week_dead_cross
    test_list['week_up'] = week_up
    test_list['week_down'] = week_down

    #if any(test_list['check']) == 1:
    rrr = rrr.append(test_list)
    print(i, v, '완료')

0 000230 완료
1 000400 완료
2 000885 완료
3 000890 완료
4 001140 완료
5 001210 완료
6 001210 완료
7 001340 완료
8 001360 완료
9 001360 완료
10 001360 완료
11 001440 완료
12 001790 완료
13 001790 완료
14 001820 완료
15 002020 완료
16 002070 완료
17 002070 완료
18 002070 완료
19 002100 완료
20 002140 완료
21 002140 완료
22 002210 완료
23 002360 완료
24 002720 완료
25 002720 완료
26 002760 완료
27 002760 완료
28 002995 완료
29 003060 완료
30 003060 완료
31 003060 완료
32 003280 완료
33 003280 완료
34 003350 완료
35 003560 완료
36 003580 완료
37 003580 완료
38 003580 완료
39 004100 완료
40 004770 완료
41 004870 완료
42 005070 완료
43 005070 완료
44 005110 완료
45 005110 완료
46 005360 완료
47 005390 완료
48 005390 완료
49 005690 완료
50 006090 완료
51 006280 완료
52 006490 완료
53 006490 완료
54 006840 완료
55 006980 완료
56 006980 완료
57 006980 완료
58 007110 완료
59 007210 완료
60 007460 완료
61 007460 완료
62 007460 완료
63 007660 완료
64 007810 완료
65 008350 완료
66 008350 완료
67 008350 완료
68 008350 완료
69 008355 완료
70 008600 완료
71 008600 완료
72 008730 완료
73 009310 완료
74 009320 완료
75 009420 완료
76 009460 완료
77 009830

594 300080 완료
595 303030 완료
596 303030 완료
597 307160 완료
598 307180 완료
599 307930 완료
600 311390 완료
601 311390 완료
602 313760 완료
603 317850 완료
604 317850 완료
605 317850 완료
606 318000 완료
607 318000 완료
608 900100 완료
609 900100 완료
610 900100 완료
611 900270 완료
612 900280 완료
613 900290 완료
614 900310 완료
615 900310 완료
616 900340 완료
617 900340 완료
618 900340 완료
619 900340 완료


In [73]:
df_for = fin_result[['index','shcode','shname','target_day']]

In [74]:
result_for_draw = pd.DataFrame([],columns = ['index','shcode','interval_init', 'target_day','interval_end'])

for i,v in df_for['index'].items():
    shcode = df_for['shcode'].iloc[i].zfill(6)
    target_date = df_for['target_day'].iloc[i]
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    #interval calculate before 4 weeks , after 2 weeks
    interval_init_timestamp = target_day_list - datetime.timedelta(weeks=25)
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=4)
    

    # make df using series(datetime to str)
    result = pd.DataFrame(dict(interval_init = [interval_init_timestamp.strftime("%Y%m%d")],
                               target_day = [target_day_list.strftime("%Y%m%d")],
                               interval_end = [interval_end_timestamp.strftime("%Y%m%d")]))
    
    result['index'] = v
    result['shcode'] = shcode
    
    result_for_draw = result_for_draw.append(result)

In [75]:
per_df = pd.DataFrame([],columns = ['date','index','shcode'])

for i in range(len(result_for_draw)):
    df = query.query_ohlcv(shcode = result_for_draw['shcode'].iloc[i],
                         fr=result_for_draw['interval_init'].iloc[i],
                         to=result_for_draw['interval_end'].iloc[i])
    
    #데이터전처리
    df = preprocesser.sma_long(df)
    df = preprocesser.obv(df)
    df = preprocesser.rsi(df)
    df = preprocesser.stochastic(df)
    df = preprocesser.marking_cross_sma(df)
    df = preprocesser.marking_cross_rsi(df)
    df = preprocesser.marking_cross_stochastic(df)
    
    #df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
    df['index'] = result_for_draw['index'].iloc[i]
    df['shcode'] = result_for_draw['shcode'].iloc[i]
    df['target_day'] = result_for_draw['target_day'].iloc[i]
    
    per_df = per_df.append(df[['date','index','shcode',
                               'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
                               'obv',
                               'rsi14', 'rsi14_signal', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
                               'stoc_g_c', 'stoc_d_c']])

In [76]:
final = pd.merge(rrr, per_df, how = 'left', on = ['date','index','shcode'])

In [77]:
final[['index' ,'shcode','shname', 'target_m', 'date', 'close_ratio', 'open_', 'low', 'high', 'close','volume' , 
       'range_position', 'squeezed',
       'month_ma5_score', 'high_month', 'for_net_buy_sum','com_net_buy_sum', 'per_net_buy_sum',
       'week_ma5_score','week_ma20_score',
       'week_golden_cross', 'week_dead_cross', 'week_up', 'week_down',
       'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
       'obv',
       'rsi14', 'rsi14_signal', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
       'stoc_g_c', 'stoc_d_c'
      ]].to_csv("bf_target_day_using_weekly_v2.csv",encoding = 'ms949')